Ce programme implémente l'algorithme génétique pour une application de conversion de monnaie dans le but de trouver le groupe de monnaie qui permettent un gain.

In [1877]:
! pip install requests --user

from collections import namedtuple
from functools import partial
from random import choices, randint, randrange,random
from typing import List,Callable,Tuple
import requests
from tkinter import *
import tkinter as tk
import json
from tkinter import ttk
import time
import csv
import numpy

Classe qui permet de retourner un object de type Currency qui permet de traiter monnaies.

In [1878]:
class Currency:
    def __init__(self, name:str,rate:float):
        self.name = name
        self.rate = rate
    def __str__(self):
        return f"{self.name}:{self.rate}"

Classe qui permet de convertir des objets de type Currency.

In [1879]:
class RealTimeCurrencyConverter():
    def __init__(self, url,name):
        self.data = requests.get(url+name).json()
        self.data.update()
        self.currencies = self.data['rates']
        self.name = name

    def convert(self, from_currency:Currency, to_currency:Currency, amount):
        initial_amount = amount
        if from_currency != self.name:
            amount = amount / self.currencies[from_currency.name]

        # limiting the precision to 4 decimal places
        amount = round(amount * self.currencies[to_currency.name], 4)
        return amount

Objets qui nous servirons lors de l'application de l'algorithme génétique.

In [1880]:
Genome = List[Currency]
Population = List[Genome]

Variables pour la liste de monnaies présentent dans le API.

In [1881]:
url = 'https://api.exchangerate-api.com/v4/latest/'
converter = RealTimeCurrencyConverter(url,"USD")
global s
s = []
for key,value in converter.currencies.items():
    a = Currency(key,value)
    s.append(a)
global things
things = s
global realGainer
realGainer = []

Fonction qui génère une liste génome aléatoirement de la liste de monnaie retourné du API.

In [1882]:
def generete_genome(length:int)->Genome:
    return choices(s, k = length)

Fonction qui génère une population de génomes sur laquelle appliqué l'algorithme.

In [1883]:
def generate_population(size:int, genome_length)->Population:
    return [generete_genome(genome_length) for _ in range(size)]

Fonction Fitness qui retourne un génome de monnaies(Currency) qui offre un gain.

In [1884]:
def fitness(genome:Genome, things = things)->Genome:
    global gainers
    gainers = []
    j = 0
    for i in genome[j:len(genome)]:
        j+=1
        for x in genome[j:len(genome)]:
            j+=1
            a = converter.convert(i,x,1)
            for c in genome[j:len(genome)]:
                j+=1
                m = converter.convert(x,c,a)
                for z in genome[j:len(genome)]:
                    j+=1
                    o = converter.convert(c,z,m)
                    for y in genome[j:len(genome)]:
                        h = converter.convert(z,y,o)
                        f = converter.convert(y,i,h)
                        if(f>i.rate):
                            #print("Adding Gainer")
                            realGainer.append([[i,x,c,z,y],f-i.rate])
                            break
                        if(f<i.rate):
                            gainers.append([i,x,c,z,y])
                            #print("Loss")
                        #if(f==genome[i].rate):
                        #    print("Nothing")
    return gainers

Fontion qui sélectionne un pair de monnaies parmi les monnaies passé.

In [1885]:
def selection_pair(population:Population):
    return choices(population,k=2)

Fonction qui croise deux génomes passé et qui retourne un duo de génomes générés pas le croisement.

In [1886]:
def single_point_crossover(a:Genome, b:Genome, popuLength:int) -> Tuple[Genome,Genome]:
    p = randint(1, popuLength)
    return a[0][0:p] + b[0][p:], b[0][0:p] + a[0][p:]

Fonction de mutation d'un génome.

In [1887]:
def mutation(genome:Genome) -> Genome:
    num = 1
    prob = 0.8
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > prob else genome[abs(index - 1)]
    return genome

Fonction qui permet d'exécuter chaque propriété de l'algorithme génétique

In [1888]:
def run_evolution(popultionSize,populationLength):
    population = generate_population(popultionSize,populationLength)
    s = []
    for i in population:
        t = fitness(i)
        if(t == []):
            population.remove(i)
        if (t != []):
            s.append(t)
    if(len(s)>=2):
        global next_generation
        next_generation = s[0:2]
        parents = selection_pair(s)
        offspring_a,offspring_b=single_point_crossover(parents[0],parents[1],len(s))
        offspring_a = mutation(offspring_a)
        offspring_b = mutation(offspring_b)
        next_generation += [offspring_a,offspring_b]
        population = next_generation
    return population

In [1889]:
def main():
    a = run_evolution(1000, 5)
    # open the file in the write mode
    print("Applying fitness to a")
    u = []
    for i in a:
        for ii in i:
            if(type(ii) == type(i)):
                r = fitness(ii)
                u.append(r)
                break
        t = fitness(i)
        if (t == []):
            print("Not fit")
            a.remove(i)
        if (t != []):
            print("Adding T")
            u.append(t)

    # print(a)
    # print(u)

    f2 = open('/CurrenciesProject/data2.csv', 'w')
    writer2 = csv.writer(f2)
    for uu in u:
        for uuu in uu:
            writer2.writerow(uuu)

    f = open('/CurrenciesProject/data.csv', 'w')

    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    
    for aa in realGainer:
        p = []
        for aaa in aa[0]:
            p.append((str(aaa)))
        writer.writerow((p,aa[1]))
        

    # close the file
    f.close()


In [1890]:
if __name__ == "__main__":
    main()

Applying fitness to a
Not fit
Adding T
Adding T
